In [25]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from joblib import dump, load
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
import swifter
from tqdm import tqdm

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [26]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [27]:
start_year = 2009
end_year = 2018
exclude = '_no_Others'

In [28]:
def predict(X, topn=3):
    preprocessed_X = preprocess_text(X)
    vec_X = doc2vec_model.infer_vector(preprocessed_X)
    vec_X = vec_X.reshape(1, vec_X.shape[0])
    pred = classifier.predict(vec_X)
    pred_i = [(topics_index_to_name_map[i], p) for i,p in enumerate(pred[0])]
    pred_sorted = sorted(pred_i, key=lambda x: x[1], reverse=True)
    return pred_sorted[:topn]

In [29]:
def build_network():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(100)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(len(enc.categories_[0]), activation='softmax')
    ])
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0, name='categorical_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

In [30]:
doc2vec_model = Doc2Vec.load('../models/doc2vec/doc2vec_{}_{}{}'.format(start_year, end_year, exclude))
enc = load('../models/doc2vec/encoder_{}_{}{}.joblib'.format(start_year, end_year, exclude))
classifier = build_network()
classifier.load_weights('../models/doc2vec/classifier_{}_{}{}'.format(start_year, end_year, exclude))

In [40]:
df = pd.read_csv('../data/2016_commons.csv')
df = df.drop(df[df.topic == 'admin'].index)
df = df.drop(['Unnamed: 0'], axis=1)
df = df.groupby(['date', 'topic'])['transcript'].apply(lambda x: ' '.join(x)).reset_index()
# df = df.drop(df[df.transcript.str.split().map(len) < 10].index).reset_index()
# df['topic'] = df.apply(lambda row: preprocess(row['topic']), axis=1)

In [41]:
df

,date,topic,transcript
0,2016-01-05,Counter-terrorism,"With permission, Mr Speaker, I will make a sta..."
1,2016-01-05,EU Council,"With permission, Mr Speaker, I would like to m..."
2,2016-01-05,Flooding,"With permission, Mr Speaker, I would like to m..."
3,2016-01-05,Future Funding for S4C,I thank the many colleagues who have stayed so...
4,2016-01-05,Health >> Children and Young People’s Mental H...,14. What steps his Department is taking to in...
...,...,...,...
2438,2016-12-20,Health >> Sustainability and Transformation Pl...,10. What assessment he has made of the potenti...
2439,2016-12-20,Health >> Topical Questions,T2. If he will make a statement on his depart...
2440,2016-12-20,Health >> Unhealthy Food,15. What assessment he has made of the potenti...
2441,2016-12-20,Leasehold and Commonhold Reform,"I beg to move, That this House has considered..."


In [42]:
X = df['transcript'].values
Y = df['topic'].values
i = 45
print(predict(X[i]))
print('true: ', Y[i], Y[i])

[('Energy and environment', 0.9329466), ('Business, industry and consumers', 0.02679499), ('Culture, media and sport', 0.014252685)]
true:  Energy and Climate Change >> Paris Agreement on Climate Change Energy and Climate Change >> Paris Agreement on Climate Change


In [43]:
def build_network():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(100)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
    ])
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0, name='categorical_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

In [44]:
X = df['transcript'].values
Y = df['topic'].values
i = 45
print(predict(X[i]))
print('true: ', Y[i], Y[i])

[('Energy and environment', 0.92720366), ('Business, industry and consumers', 0.028068416), ('Culture, media and sport', 0.014360879)]
true:  Energy and Climate Change >> Paris Agreement on Climate Change Energy and Climate Change >> Paris Agreement on Climate Change


In [46]:
y_pred = []
for x in tqdm(X):
    y_pred.append(predict(x)[0][0])
df['pred'] = y_pred

100%|██████████| 2443/2443 [09:50<00:00,  4.13it/s]


In [ ]:
df

In [47]:
df.to_csv('2016_commons_pred.csv')